# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Lab 07 - Real-time log analyzer** </center>
---
## <center> **Big Data** </center>
---
### <center> **Spring 2025** </center>
---
### <center> **03/31/2025** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

**Team**: Foraneos

**Students**: Eddie, Konrad 

In [ ]:
import findspark
findspark.init()

#### Spark Session creation


In [2]:
from pyspark.sql import SparkSession

konrad_port = ""
eddie_port = "8776010e8f6a"

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Files") \
    .master("spark://{}:7077".format(eddie_port)) \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/03 02:04:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### File stream initialization

In [17]:
entries = spark \
                .readStream \
                .format("text") \
                .option("cleanSource", "archive") \
                .option("sourceArchiveDir", "/home/jovyan/archive") \
                .load("/home/jovyan/notebooks/data/strucutred_streaming_files/")

### Set the output sink of the stream

In [ ]:
from pyspark.sql.functions import split, col, count

# Parse the log data into separate columns
entries = entries.withColumn("data", split(col("value"), " \\| ")) \
                 .withColumn("timestamp", col("data").getItem(0)) \
                 .withColumn("type", col("data").getItem(1)) \
                 .withColumn("message", col("data").getItem(2)) \
                 .withColumn("node", col("data").getItem(3)) \
                 .drop("value", "data") \
                 .filter(col("type") == "ERROR")

# Count the total number of ERROR entries
len_entries = entries.groupBy().agg(count("*").alias("total_error_entries"))

# Stream the error log entries to the console
query_entries = entries \
                .writeStream \
                .outputMode("append") \
                .trigger(processingTime='2 seconds') \
                .format("console") \
                .start()

query_entries.awaitTermination()

# Uncomment below to stream the count of error entries
# query_len_entries = len_entries \
#                 .writeStream \
#                 .outputMode("complete") \
#                 .trigger(processingTime='2 seconds') \
#                 .format("console") \
#                 .start()

# query_len_entries.awaitTermination()

25/04/03 02:12:46 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-bd2a3268-8ca6-4be1-8f20-bcc1a07f19da. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/04/03 02:12:46 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+---------+----+-------+----+
|timestamp|type|message|node|
+---------+----+-------+----+
+---------+----+-------+----+

-------------------------------------------
Batch: 1
-------------------------------------------
+---------+----+-------+----+
|timestamp|type|message|node|
+---------+----+-------+----+
+---------+----+-------+----+

-------------------------------------------
Batch: 2
-------------------------------------------
+---------+----+-------+----+
|timestamp|type|message|node|
+---------+----+-------+----+
+---------+----+-------+----+

-------------------------------------------
Batch: 3
-------------------------------------------
+---------+----+-------+----+
|timestamp|type|message|node|
+---------+----+-------+----+
+---------+----+-------+----+

-------------------------------------------
Batch: 4
-------------------------------------------
+---------+----+-------+----+
|

25/04/03 02:20:06 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 2000 milliseconds, but spent 2159 milliseconds


-------------------------------------------
Batch: 37
-------------------------------------------
+---------+----+-------+----+
|timestamp|type|message|node|
+---------+----+-------+----+
+---------+----+-------+----+

-------------------------------------------
Batch: 38
-------------------------------------------
+---------+----+-------+----+
|timestamp|type|message|node|
+---------+----+-------+----+
+---------+----+-------+----+

-------------------------------------------
Batch: 39
-------------------------------------------
+---------+----+-------+----+
|timestamp|type|message|node|
+---------+----+-------+----+
+---------+----+-------+----+

-------------------------------------------
Batch: 40
-------------------------------------------
+---------+----+-------+----+
|timestamp|type|message|node|
+---------+----+-------+----+
+---------+----+-------+----+

-------------------------------------------
Batch: 41
-------------------------------------------
+---------+----+-------+--

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

-------------------------------------------
Batch: 70
-------------------------------------------
+---------+----+-------+----+
|timestamp|type|message|node|
+---------+----+-------+----+
+---------+----+-------+----+

-------------------------------------------
Batch: 71
-------------------------------------------
+---------+----+-------+----+
|timestamp|type|message|node|
+---------+----+-------+----+
+---------+----+-------+----+

-------------------------------------------
Batch: 72
-------------------------------------------
+---------+----+-------+----+
|timestamp|type|message|node|
+---------+----+-------+----+
+---------+----+-------+----+

-------------------------------------------
Batch: 73
-------------------------------------------
+---------+----+-------+----+
|timestamp|type|message|node|
+---------+----+-------+----+
+---------+----+-------+----+

-------------------------------------------
Batch: 74
-------------------------------------------
+---------+----+-------+--

In [16]:
query_entries.stop()
# query_len_entries.stop()

In [24]:
sc.stop()